In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import linregress

### Langevin Equation
$$ \gamma_0 \dot{x_1} =  a_{00} x_1 + a_{01} x_2  + \sqrt{2k_B T \gamma_0}ξ_0 $$
$$ \gamma_1 \dot{x_2} =  a_{10} x_1 + a_{11} x_2  + \sqrt{2k_B T \gamma_1}ξ_1 $$

Which we can rewrite as:

$$ \dot{x_1} = \frac{a_{00}}{\gamma_0} x_1 + \frac{a_{01}}{\gamma_0} x_2 + \sqrt{\frac{2k_B T_1 \gamma_0}{\gamma_0^2}}ξ_0 $$
$$ \dot{x_2} =  \frac{a_{10}}{\gamma_1} x_1 + \frac{a_{11}}{\gamma_1} x_2 + \sqrt{\frac{2k_B T_2 \gamma_1}{\gamma_1^2}}ξ_1 $$

where: $a_{00} = -1$, $a_{01} = -0.5$, $a_{10} = -0.5$, $a_{11} = -1$, $k_B T = 4.1$, $\gamma_0 = 1$, $\gamma_1 = 1$

### Euler method

Euler Scheme

$$ X_{t+\Delta t} = X_t + f(X_t, t)\Delta t + g(X_t,t)\Delta B_t $$

where $$ \Delta B_t = \mathcal{N}(0, \sqrt\Delta t) $$

In [2]:
#paramaters
#change all units to: force = pN, length = nm, time = ms
pars = {}
pars['a00'] = -1
pars['a01'] = -0
pars['a10'] = -0.5
pars['a11'] = -1
pars['gamma0'] = 1
pars['gamma1'] = 1

pars['kbT'] = 4.1
pars['noise0'] = 2*pars['gamma0']*pars['kbT']
pars['noise1'] = 2*pars['gamma1']*pars['kbT']

pars['delta_t'] = 0.01
pars['n_realizations'] = 100

In [3]:
def euler_one_traj(n_time, pars):   
    #initial condition
    x1 = 0
    x2 = 0
    x1_traj = []
    x2_traj = []
    
    #performing 1 traj with n_time number of data points
    for i in range(n_time):
        #generate delta_B
        s1 = np.random.normal(0, np.sqrt(pars['delta_t']) )
        s2 = np.random.normal(0, np.sqrt(pars['delta_t']) )
        #solve for x1, x2
        x1_new = x1 + (pars['a00']/pars['gamma0'])*x1*pars['delta_t'] + \
        (pars['a01']/pars['gamma0'])*x2*pars['delta_t'] + \
        np.sqrt((pars['noise0'])/((pars['gamma0'])**2))*s1
                 
        x2_new = x2 + (pars['a11']/pars['gamma1'])*x2*pars['delta_t'] + \
        (pars['a10']/pars['gamma1'])*x1*pars['delta_t'] + \
        np.sqrt((pars['noise1'])/((pars['gamma0'])**2))*s2
                
        x1 = x1_new
        x2 = x2_new
        
        x1_traj.append(x1)
        x2_traj.append(x2)
    return x1_traj, x2_traj

### Functions for stochastic heat

### Stochastic Heat:

$$ 0 = - \gamma_0 \dot{x_1} + \sqrt{2k_B T \gamma_0}ξ_0 + a_{00} x_1 + a_{01} x_2  $$
$$ 0 = - \gamma_1 \dot{x_2} + \sqrt{2k_B T \gamma_1}ξ_1 + a_{10} x_1 + a_{11} x_2   $$

For $x_1$:

$$ \delta Q_1 = (-\gamma_0 \dot{x} + \sqrt{2k_B T \gamma_0}ξ_0) \circ dx_1  = -(a_{00} x_1 + a_{01} x_2) \circ dx_1$$

$$ \delta Q_1 = -(a_{00} x_1 + a_{01} x_2) \circ dx_1 $$

For $x_2$:

$$ \delta Q_2 = (-\gamma_1 \dot{x} + \sqrt{2k_B T \gamma_1}ξ_1) \circ dx_2  = -(a_{10} x_1 + a_{11} x_2) \circ dx_2$$

$$ \delta Q_2 = -(a_{10} x_1 + a_{11} x_2) \circ dx_2 $$

Final equation for stochastic heat:

$$ \delta Q_1 = -(a_{00} x_1 + a_{01} x_2) \circ dx_1  $$

$$ \delta Q_2 = -(a_{11} x_2 + a_{10} x_1) \circ dx_2  $$

In [4]:
def calc_stochastic_heat_q1(x1, x2, pars):
    """Calculates stochastic heat per time delta following equation (3)
    Input:
    x1 (list) = x1_trajectory_list
    x2 (list) = x2_trajectory_list
    pars (dictionary) = parameters
    
    Here's an example:
    x1 =       [1,2,3,4,5]
    x1_shift = [5,1,2,3,4]
    Then we can easily calculate operations, e.g. dx1 = x1 - x1_ shift
    Vectorization (technique of implementing array operations without using for loop) is easier.
    
    After which, we discard the first element
    
    Returns:
    stochastic heat as shown in equation (3)"""
    x1_shift = np.roll(x1, shift = 1)   
    x2_shift = np.roll(x2, shift = 1)
    
    first_term = -pars['a00']*((x1 + x1_shift)/2)
    second_term = -pars['a01']*((x2 + x2_shift)/2)
    dx1 = x1 - x1_shift
    return (first_term[1:] + second_term[1:])*dx1[1:]

def calc_stochastic_heat_q2(x1, x2, pars):
    x1_shift = np.roll(x1, shift = 1)   
    x2_shift = np.roll(x2, shift = 1)    
    
    first_term = -pars['a11']*((x2 + x2_shift)/2)
    second_term = -pars['a10']*((x1 + x1_shift)/2)
    dx2 = x2 - x2_shift
    return (second_term[1:] + first_term[1:])*dx2[1:]

In [5]:
#function to calculate average q (cumulative_heat) after performing n_realizations
def average_q(max1_time, pars):
    min_time = 0
    n_time = int((max1_time - min_time)/pars['delta_t'] + 1) #n_time = num of evenlx2 spaced samples (ex1. 1001 points)
    time = np.linspace(min_time, max1_time, n_time) #time(ex1. 0 (min time), 0.01, 0.02, ..., 10 (max1 time)) with n_time no. of data points
    
    average_q1_trajectorx2 = 0
    average_q2_trajectorx2 = 0
    
    relax_time = int(abs(pars['gamma0']/pars['a00']))
    
    for i in range(pars['n_realizations']):
        x1_traj, x2_traj = euler_one_traj(n_time, pars)
        final_x1_traj = x1_traj[relax_time::]
        final_x2_traj = x2_traj[relax_time::]
        
        
        q1_trajectorx2 = calc_stochastic_heat_q1(final_x1_traj, final_x2_traj, pars)
        q1_cumulative = np.cumsum(q1_trajectorx2)
        
        q2_trajectorx2 = calc_stochastic_heat_q2(final_x1_traj, final_x2_traj, pars)
        q2_cumulative = np.cumsum(q2_trajectorx2)
        
        average_q1_trajectorx2 += q1_cumulative/pars['n_realizations']
        average_q2_trajectorx2 += q2_cumulative/pars['n_realizations']
        
    return average_q1_trajectorx2, average_q2_trajectorx2, time[:-1]

In [ ]:
a01_list = np.arange(0, 0.5 + 0.01, 0.01)

qx1_last_element_list = []
qx1_slope_list = []
for i in tqdm(a01_list):
    pars['a01'] = -i
    qx1_neq, qx2_neq, time = average_q(300, pars)
    last_element = qx1_neq[-1]
    qx1_last_element_list.append(last_element)
    
    x_axis = np.linspace(0,len(qx1_neq), len(qx1_neq))
    result_qx1_neq = linregress(x_axis, qx1_neq)
    qx1_slope_list.append(result_qx1_neq.slope)
    

 22%|██▏       | 11/51 [07:16<25:07, 37.68s/it]

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.scatter(a01_list, qx1_last_element_list)

plt.xlabel('$a_{01}$', fontsize=12.5)
plt.ylabel('$\\left<\dot{Q}_t \\right>$ ($pN$ $nm$)', fontsize=12.5)
plt.xticks(fontsize=12.5)
plt.yticks(fontsize=12.5)
plt.savefig('reciprocity_q.png', dpi=300)

plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.scatter(a01_list, qx1_slope_list)
plt.ylabel('slope of average cumulative heat')
plt.xlabel('$a_{01}$')

plt.show()